In [284]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [285]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_layer_dimensions, hidden_layer_dimensions, output_layer_dimensions):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_layer_dimensions, hidden_layer_dimensions),
            nn.ReLU(),
            nn.Linear(hidden_layer_dimensions, output_layer_dimensions),
            nn.Softmax()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork(13, 28, 2)
print(model)

data_path = "/home/itibcn/Desktop/Torch/heart.csv"

#x
data = np.genfromtxt( 
    data_path,
    dtype=np.float32, 
    skip_header=True,
    delimiter=",")

X = torch.tensor(data[:,:-1], dtype=torch.float32).to(device)#Tots els valors menos l'ultima columna
y_real = torch.tensor(data[:,-1], dtype=torch.long).to(device) #Nomes l'ultima fila

tensor = torch.tensor(data)

logits = model(X)

split_idx = int(len(X) * 0.7)
train = TensorDataset(X[:split_idx], y_real[:split_idx])
test = TensorDataset(X[split_idx:], y_real[split_idx:])


pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

print(f"Class predita:  {y_pred}")

print("pred_probab: ")
print(pred_probab)

#print(tensor)
print(tensor.shape)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=13, out_features=28, bias=True)
    (1): ReLU()
    (2): Linear(in_features=28, out_features=2, bias=True)
    (3): Softmax(dim=None)
  )
)
Class predita:  tensor([0, 0, 0,  ..., 0, 0, 0])
pred_probab: 
tensor([[0.7311, 0.2689],
        [0.7311, 0.2689],
        [0.7311, 0.2689],
        ...,
        [0.7289, 0.2711],
        [0.7308, 0.2692],
        [0.7310, 0.2690]], grad_fn=<SoftmaxBackward0>)
torch.Size([1025, 14])


In [290]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

train_dataloader = DataLoader(train, batch_size, True)
test_dataloader = DataLoader(train, batch_size, True)


for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    
print("Done!")

learning_rate = 1e-3
batch_size = 64
epochs = 10

Epoch 1
-------------------------------
loss: 0.757963  [   64/  717]
Test Error: 
 Accuracy: 70.4%, Avg loss: 0.609322 

Epoch 2
-------------------------------
loss: 0.664115  [   64/  717]
Test Error: 
 Accuracy: 70.6%, Avg loss: 0.577722 

Epoch 3
-------------------------------
loss: 0.530758  [   64/  717]
Test Error: 
 Accuracy: 56.2%, Avg loss: 0.726442 

Epoch 4
-------------------------------
loss: 0.751740  [   64/  717]
Test Error: 
 Accuracy: 71.4%, Avg loss: 0.588366 

Epoch 5
-------------------------------
loss: 0.601442  [   64/  717]
Test Error: 
 Accuracy: 62.1%, Avg loss: 0.643660 

Done!


Train loop / Test loop

In [182]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")


Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=13, out_features=28, bias=True)
    (1): ReLU()
    (2): Linear(in_features=28, out_features=2, bias=True)
    (3): Softmax(dim=None)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([28, 13]) | Values : tensor([[-0.1158,  0.1634, -0.1528,  0.1071,  0.2142,  0.0643, -0.2701, -0.1284,
          0.2181, -0.2519, -0.0949,  0.0674, -0.2325],
        [-0.1113,  0.0340, -0.0605, -0.0634, -0.0628,  0.1455,  0.2551, -0.1101,
         -0.1015, -0.1997,  0.2629, -0.0891, -0.0007]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([28]) | Values : tensor([-0.0240,  0.1946], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([2, 28]) | Values : tensor([[ 0.1775,  0.1042,  0.0345,  0.0980,  0.1808,  0.1198,  0.0805,  0.0330,
          0.1478, -0.1074, -0.1577,  0.0285,  0.1414, -0.0

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=13, out_features=28, bias=True)
    (1): ReLU()
    (2): Linear(in_features=28, out_features=2, bias=True)
    (3): Softmax(dim=None)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([28, 13]) | Values : tensor([[ 0.0796,  0.0650,  0.0282,  0.0006, -0.1855, -0.2478, -0.0708,  0.1813,
         -0.2550, -0.1493, -0.1740,  0.1298,  0.0315],
        [-0.1111,  0.0077, -0.1406,  0.1204, -0.2201, -0.1952,  0.1806, -0.1399,
          0.0333,  0.1400, -0.2449, -0.1098,  0.0324]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([28]) | Values : tensor([-0.2665, -0.0542], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([2, 28]) | Values : tensor([[-0.1808, -0.0330, -0.1316, -0.0374,  0.0909,  0.1847, -0.0426, -0.0696,
          0.1885, -0.1601, -0.0419, -0.0740, -0.1581,  0.1